In [1]:
import torch
import sys
from tqdm import tqdm

In [2]:
from model import get_stage

whole_model_dict = torch.load("../pyllama/whole_llama1_model.pt")
whole_model_keys_lst = list(whole_model_dict.keys())

In [3]:
for k,v in whole_model_dict.items():
    print(k, v.shape)

tok_embeddings.weight torch.Size([32000, 6656])
norm.weight torch.Size([6656])
output.weight torch.Size([32000, 6656])
layers.0.attention.wq.weight torch.Size([6656, 6656])
layers.0.attention.wk.weight torch.Size([6656, 6656])
layers.0.attention.wv.weight torch.Size([6656, 6656])
layers.0.attention.wo.weight torch.Size([6656, 6656])
layers.0.feed_forward.w1.weight torch.Size([17920, 6656])
layers.0.feed_forward.w2.weight torch.Size([6656, 17920])
layers.0.feed_forward.w3.weight torch.Size([17920, 6656])
layers.0.attention_norm.weight torch.Size([6656])
layers.0.ffn_norm.weight torch.Size([6656])
layers.1.attention.wq.weight torch.Size([6656, 6656])
layers.1.attention.wk.weight torch.Size([6656, 6656])
layers.1.attention.wv.weight torch.Size([6656, 6656])
layers.1.attention.wo.weight torch.Size([6656, 6656])
layers.1.feed_forward.w1.weight torch.Size([17920, 6656])
layers.1.feed_forward.w2.weight torch.Size([6656, 17920])
layers.1.feed_forward.w3.weight torch.Size([17920, 6656])
layers.

In [4]:
model_lst = []
for i in range(4):
    model_lst.append(getattr(get_stage, "model_"+str(i))())
model_lst

[model_0(
   (tok_embeddings): VocabEmbedding()
   (layers): ModuleList(
     (0-14): 15 x TransformerBlock(
       (attention): Attention(
         (wq): noParallelLinear()
         (wk): noParallelLinear()
         (wv): noParallelLinear()
         (wo): noParallelLinear()
       )
       (feed_forward): FeedForward(
         (w1): noParallelLinear()
         (w2): noParallelLinear()
         (w3): noParallelLinear()
       )
       (attention_norm): RMSNorm()
       (ffn_norm): RMSNorm()
     )
   )
 ),
 model_1(
   (layers): ModuleList(
     (0-14): 15 x TransformerBlock(
       (attention): Attention(
         (wq): noParallelLinear()
         (wk): noParallelLinear()
         (wv): noParallelLinear()
         (wo): noParallelLinear()
       )
       (feed_forward): FeedForward(
         (w1): noParallelLinear()
         (w2): noParallelLinear()
         (w3): noParallelLinear()
       )
       (attention_norm): RMSNorm()
       (ffn_norm): RMSNorm()
     )
   )
 ),
 model_2(
   (

In [5]:
for i in range(4):
    print(i)
    state_dict = model_lst[i].state_dict()
    new_state_dict = {}
    for k, v in state_dict.items():
        if "layers" in k:
            new_k = k.lstrip("layers.")
            layer_id = int(new_k.split(".")[0])
            tmp_k = new_k.lstrip(str(layer_id))
            layer_id += 15*(i)
            new_k = "layers." + str(layer_id) + tmp_k
            new_state_dict[k] = whole_model_dict[new_k]
        else:
            new_state_dict[k] = whole_model_dict[k]
    model_lst[i].load_state_dict(new_state_dict)            

0
1
2
3


In [7]:
for i, model_ in enumerate(tqdm(model_lst)):
    torch.save(model_.state_dict(), "pp_ckpt/stage_"+str(i)+".pt")

100%|██████████| 4/4 [07:28<00:00, 112.21s/it]
